In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
plt.style.use(['science','notebook', 'grid'])
import numba
from numba import njit
from scipy.ndimage import convolve, generate_binary_structure

In [ ]:
#form a square lattice size N with half spins up and half down randomly distributed 
def lattice(N):
    lattice = np.zeros((N, N))
    random = np.random.random((N, N))
    lattice[random > 0.5 ] = 1
    lattice[random <= 0.5 ] = -1
    return lattice

#calculate energy of a given lattice with help of convolutin
def energy(lattice): 
    ker = generate_binary_structure(2,1)
    ker[1][1] = False
    E =  - (lattice * convolve(lattice, ker, mode='wrap', cval=0)).sum() /2
    return E

#calculate magnetisation of a given lattice by suming all the spin values 
def magnetisation(lattice):
    return lattice.sum()

In [ ]:
#metropolis algorithm that takes square lattice, size value N, number of MCS/per spin marked as st,
#"temperature" of a system BJ, its energy E and magnetisation M
#makeing calcualtions faster by using numba.njit library 
@numba.njit(nopython=True)
def metropolis(lattice, N, st, BJ, E, M): 
    lattice = lattice.copy()
    net_spins = np.zeros(st-1)
    net_energy = np.zeros(st-1)
    for a in range(0,st-1):   #nuber of MCS/spin
        for b in range(N**2): #one MCS/spin
            i = random.randint(0,N-1)
            j = random.randint(0,N-1)
            initial = lattice[i][j] #choose random spin 
            neigbours = lattice[i][(j-1)%N] + lattice[(i-1)%N][j] + lattice[i][(j+1)%N] + lattice[(i+1)%N][j]
            delta_E =  2 *  initial * neigbours #calculate energy diferance before and after spin flip 
            if delta_E <= 0: #if fliped spin position resolves in lower enrgy  flip the spin
                lattice[i][j] = (-1) * initial 
                E += delta_E
                M -= initial*2
            elif (delta_E > 0) * (random.random() < np.exp(- BJ * delta_E )): 
                lattice[i][j] = (-1) * initial   #if fliped spin increases total enegy than flip the spin by some propability 
                E += delta_E
                M -= initial*2
        net_spins[a] = abs(M)
        net_energy[a] = E
    return lattice, net_energy, net_spins


In [ ]:
#function for calculating values needed for average magnetisation, specific heat, magnetic susceptibility and two comulants
#function takes as na argument square lattice, "temperatures" BJ at which we are interested in the calculated values,
# N size of imputed lattice and nuber of MCS/per spin. 
def get_m_E(lattice, BJs, N, st):

    lattice = lattice.copy()
    mean_spin = np.zeros(len(BJs))
    mean_spin_squared = np.zeros(len(BJs))
    mean_spin_na4 = np.zeros(len(BJs))
    var_spin = np.zeros(len(BJs))
    mean_E = np.zeros(len(BJs))
    mean_E_squared = np.zeros(len(BJs))
    var_E = np.zeros(len(BJs))
    mean_mE = np.zeros(len(BJs))
    mean_m2E = np.zeros(len(BJs))

    #termalizacija prve
    arr, energies, spins = metropolis(lattice, N, 100000, BJs[0], energy(lattice), magnetisation(lattice))
   
    for count, value in enumerate(BJs):
        arr1, energies1, spins1 = metropolis(arr, N, st, value, energies[-1], magnetisation(arr))
        arr = arr1
        energies = energies1 
        spins = spins1
        mean_spin[count] = (spins[-round((0.5*st)): ][::75]/(N**2)).mean()
        mean_spin_squared[count] = ((spins[-round((0.5*st)): ][::75]/(N**2))**2).mean()
        mean_spin_na4[count] = ((spins[-round((0.5*st)): ][::75]/(N**2))**4).mean()
        var_spin[count] = spins[-round((0.5*st)): ][::75].var()/N**2

        mean_E[count] = (energies[-round((0.5*st)): ][::75]/(N**2)).mean()
        mean_E_squared[count] = ((energies[-round((0.5*st)): ][::75]/(N**2))).mean()
        var_E[count] = ((energies[-round(0.5*st): ][::75])).var()/N**2
        
        mean_mE[count] = (((energies[-round((0.5*st)): ][::75]/(N**2)))*(spins[-round((0.5*st)): ][::75]/(N**2))).mean()
        mean_m2E[count] = ((spins[-round((0.5*st)): ][::75]/(N**2))**2 * ((energies[-round((0.5*st)): ][::75]/(N**2)))).mean()
        np.savetxt('lattice_m_E_working', np.column_stack([BJs, mean_spin, mean_spin_squared, mean_spin_na4, var_spin, mean_E, mean_E_squared, var_E,  mean_mE, mean_m2E]), header='BJs mean_spin, mean_spin_squared, mean_spin_na4, var_spin, mean_E, mean_E_squared, var_E,  mean_mE, mean_m2E')
    return(mean_spin, mean_spin_squared, mean_spin_na4, var_spin, mean_E, mean_E_squared, var_E, mean_mE, mean_m2E)

In [ ]:
#create .txt files of calculated values neded for average magnetisation, specific heat, magnetic susceptibility and two comulants
#for "temperatures" in range BJ \in (0.4166667, 0.45045045) with step 0.0005

BJs = np.arange(0.4166667, 0.45045045, 0.0005)
mean_spin_10, mean_spin_squared_10, mean_spin_na4_10, var_spin_10, mean_E_10, mean_E_squared_10, var_E_10, mean_mE_10, mean_m2E_10 = get_m_E_adapted(lattice(10), BJs, 10, 100000)
np.savetxt('lattice_10_m_E', np.column_stack([BJs, mean_spin_10, mean_spin_squared_10, mean_spin_na4_10, var_spin_10, mean_E_10, mean_E_squared_10, var_E_10,  mean_mE_10, mean_m2E_10]), header='BJs mean_spin_10, mean_spin_squared_10, mean_spin_na4_10, var_spin_10, mean_E_10, mean_E_squared_10, var_E_10,  mean_mE_10, mean_m2E_10 za 100 000 MCS/S')

mean_spin_30, mean_spin_squared_30, mean_spin_na4_30, var_spin_30, mean_E_30, mean_E_squared_30, var_E_30, mean_mE_30, mean_m2E_30 = get_m_E_adapted(lattice(30), BJs, 30, 100000)
np.savetxt('lattice_30_m_E', np.column_stack([BJs, mean_spin_30, mean_spin_squared_30, mean_spin_na4_30, var_spin_30, mean_E_30, mean_E_squared_30, var_E_30, mean_mE_30, mean_m2E_30]), header='BJs, mean_spin_30, mean_spin_squared_30, mean_spin_na4_30, var_spin_30, mean_E_30, mean_E_squared_30, var_E_30, mean_mE_30, mean_m2E_30 za  100 000 MCS/S')

mean_spin_50, mean_spin_squared_50, mean_spin_na4_50, var_spin_50, mean_E_50, mean_E_squared_50, var_E_50, mean_mE_50, mean_m2E_50 = get_m_E_adapted(lattice(50), BJs, 50, 100000)
np.savetxt('lattice_50_m_E', np.column_stack([BJs, mean_spin_50, mean_spin_squared_50, mean_spin_na4_50, var_spin_50, mean_E_50, mean_E_squared_50, var_E_50, mean_mE_50, mean_m2E_50]), header='BJs, mean_spin_50, mean_spin_squared_50, mean_spin_na4_50, var_spin_50, mean_E_50, mean_E_squared_50, var_E_50, mean_mE_50, mean_m2E_50 za  100 000 MCS/S')

mean_spin_80, mean_spin_squared_80, mean_spin_na4_80, var_spin_80, mean_E_80, mean_E_squared_80, var_E_80, mean_mE_80, mean_m2E_80 = get_m_E_adapted(lattice(80), BJs, 80, 100000)
np.savetxt('lattice_80_m_E', np.column_stack([BJs, mean_spin_80, mean_spin_squared_80, mean_spin_na4_80, var_spin_80, mean_E_80, mean_E_squared_80, var_E_80, mean_mE_80, mean_m2E_80]), header='BJs, mean_spin_80, mean_spin_squared_80, mean_spin_na4_80, var_spin_80, mean_E_80, mean_E_squared_80, var_E_80, mean_mE_80, mean_m2E_80 za  100 000 MCS/S')

mean_spin_100, mean_spin_squared_100, mean_spin_na4_100, var_spin_100, mean_E_100, mean_E_squared_100, var_E_100, mean_mE_100, mean_m2E_100 = get_m_E(lattice(100), BJs, 100, 250000)
np.savetxt('lattice_100_m_E', np.column_stack([BJs, mean_spin_100, mean_spin_squared_100, mean_spin_na4_100, var_spin_100, mean_E_100, mean_E_squared_100, var_E_100, mean_mE_100, mean_m2E_100]), header='BJs, mean_spin_100, mean_spin_squared_100, mean_spin_na4_100, var_spin_100, mean_E_100, mean_E_squared_100, var_E_100, mean_mE_100, mean_m2E_100 za  200 000 MCS/S')

mean_spin_120, mean_spin_squared_120, mean_spin_na4_120, var_spin_120, mean_E_120, mean_E_squared_120, var_E_120, mean_mE_120, mean_m2E_120 = get_m_E(lattice(120), BJs, 120, 250000)
np.savetxt('lattice_120_m_E', np.column_stack([BJs, mean_spin_120, mean_spin_squared_120, mean_spin_na4_120, var_spin_120, mean_E_120, mean_E_squared_120, var_E_120, mean_mE_120, mean_m2E_120]), header='BJs, mean_spin_120, mean_spin_squared_120, mean_spin_na4_120, var_spin_120, mean_E_120, mean_E_squared_120, var_E_120, mean_mE_120, mean_m2E_120 za  100 000 MCS/S')


In [ ]:
#import previously calculated values for lattices of size 10, 30, 50, 80, 100, 120
BJs_10_peak, mean_spin_10_peak, mean_spin_squared_10_peak, mean_spin_na4_10_peak, var_spin_10_peak, mean_E_10_peak, mean_E_squared_10_peak, var_E_10_peak, mean_mE_10_peak, mean_m2E_10_peak = np.loadtxt("lattice_10_m_E")[:, 0], np.loadtxt("lattice_10_m_E")[:, 1], np.loadtxt("lattice_10_m_E")[:, 2], np.loadtxt("lattice_10_m_E")[:, 3], np.loadtxt("lattice_10_m_E")[:, 4], np.loadtxt("lattice_10_m_E")[:, 5], np.loadtxt("lattice_10_m_E")[:, 6], np.loadtxt("lattice_10_m_E")[:, 7], np.loadtxt("lattice_10_m_E")[:, 8], np.loadtxt("lattice_10_m_E")[:, 9]
BJs_30_peak, mean_spin_30_peak, mean_spin_squared_30_peak, mean_spin_na4_30_peak, var_spin_30_peak, mean_E_30_peak, mean_E_squared_30_peak, var_E_30_peak, mean_mE_30_peak, mean_m2E_30_peak = np.loadtxt("lattice_30_m_E")[:, 0], np.loadtxt("lattice_30_m_E")[:, 1], np.loadtxt("lattice_30_m_E")[:, 2], np.loadtxt("lattice_30_m_E")[:, 3], np.loadtxt("lattice_30_m_E")[:, 4], np.loadtxt("lattice_30_m_E")[:, 5], np.loadtxt("lattice_30_m_E")[:, 6], np.loadtxt("lattice_30_m_E")[:, 7], np.loadtxt("lattice_30_m_E")[:, 8], np.loadtxt("lattice_30_m_E")[:, 9]
BJs_50_peak, mean_spin_50_peak, mean_spin_squared_50_peak, mean_spin_na4_50_peak, var_spin_50_peak, mean_E_50_peak, mean_E_squared_50_peak, var_E_50_peak, mean_mE_50_peak, mean_m2E_50_peak = np.loadtxt("lattice_50_m_E")[:, 0], np.loadtxt("lattice_50_m_E")[:, 1], np.loadtxt("lattice_50_m_E")[:, 2], np.loadtxt("lattice_50_m_E")[:, 3], np.loadtxt("lattice_50_m_E")[:, 4], np.loadtxt("lattice_50_m_E")[:, 5], np.loadtxt("lattice_50_m_E")[:, 6], np.loadtxt("lattice_50_m_E")[:, 7], np.loadtxt("lattice_50_m_E")[:, 8], np.loadtxt("lattice_50_m_E")[:, 9]
BJs_80_peak, mean_spin_80_peak, mean_spin_squared_80_peak, mean_spin_na4_80_peak, var_spin_80_peak, mean_E_80_peak, mean_E_squared_80_peak, var_E_80_peak, mean_mE_80_peak, mean_m2E_80_peak = np.loadtxt("lattice_80_m_E")[:, 0], np.loadtxt("lattice_80_m_E")[:, 1], np.loadtxt("lattice_80_m_E")[:, 2], np.loadtxt("lattice_80_m_E")[:, 3], np.loadtxt("lattice_80_m_E")[:, 4], np.loadtxt("lattice_80_m_E")[:, 5], np.loadtxt("lattice_80_m_E")[:, 6], np.loadtxt("lattice_80_m_E")[:, 7], np.loadtxt("lattice_80_m_E")[:, 8], np.loadtxt("lattice_80_m_E")[:, 9]
BJs_100_peak, mean_spin_100_peak, mean_spin_squared_100_peak, mean_spin_na4_100_peak, var_spin_100_peak, mean_E_100_peak, mean_E_squared_100_peak, var_E_100_peak, mean_mE_100_peak, mean_m2E_100_peak = np.loadtxt("lattice_100_m_E")[:, 0], np.loadtxt("lattice_100_m_E")[:, 1], np.loadtxt("lattice_100_m_E")[:, 2], np.loadtxt("lattice_100_m_E")[:, 3], np.loadtxt("lattice_100_m_E")[:, 4], np.loadtxt("lattice_100_m_E")[:, 5], np.loadtxt("lattice_100_m_E")[:, 6], np.loadtxt("lattice_100_m_E")[:, 7], np.loadtxt("lattice_100_m_E")[:, 8], np.loadtxt("lattice_100_m_E")[:, 9]
BJs_120_peak, mean_spin_120_peak, mean_spin_squared_120_peak, mean_spin_na4_120_peak, var_spin_120_peak, mean_E_120_peak, mean_E_squared_120_peak, var_E_120_peak, mean_mE_120_peak, mean_m2E_120_peak = np.loadtxt("lattice_120_m_E")[:, 0], np.loadtxt("lattice_120_m_E")[:, 1], np.loadtxt("lattice_120_m_E")[:, 2], np.loadtxt("lattice_120_m_E")[:, 3], np.loadtxt("lattice_120_m_E")[:, 4], np.loadtxt("lattice_120_m_E")[:, 5], np.loadtxt("lattice_120_m_E")[:, 6], np.loadtxt("lattice_120_m_E")[:, 7], np.loadtxt("lattice_120_m_E")[:, 8], np.loadtxt("lattice_120_m_E")[:, 9]

In [ ]:
#plot average magnetic susceptibility
plt.plot(1/BJs_10_peak, BJs_10_peak* var_spin_10_peak, "o", color='red', label='10x10')
plt.plot(1/BJs_30_peak, BJs_30_peak* var_spin_30_peak, "o", color='blue', label='30x30')
plt.plot(1/BJs_50_peak, BJs_50_peak * var_spin_50_peak, "o", color='green', label='50x50')
plt.plot(1/BJs_80_peak, BJs_80_peak * var_spin_80_peak, "o", color='cyan', label='80x80')
plt.plot(1/BJs_100_peak, BJs_100_peak * var_spin_100_peak, "o", color='magenta', label='100x100')
plt.plot(1/BJs_120_peak, BJs_120_peak * var_spin_120_peak, "o", color='black', label='120x120')
plt.xlabel(r'$T$')
plt.ylabel(r'$\chi$')
plt.yscale('log')
plt.legend()
#plt.savefig('X_peak')
plt.show()
plt.close()

#find peak points for magnetisation for different lattice sizes
a_10 = (BJs_10_peak * var_spin_10_peak)
print(max(a_10))
print(BJs_10_peak[np.where(a_10 == max(a_10))])

a_30 = (BJs_30_peak * var_spin_30_peak)
print(max(a_30))
print(BJs_30_peak[np.where(a_30 == max(a_30))])

a_50 = (BJs_50_peak * var_spin_50_peak)
print(max(a_50))
print(BJs_50_peak[np.where(a_50 == max(a_50))])

a_80 = (BJs_80_peak * var_spin_80_peak)
print(max(a_80))
print(BJs_80_peak[np.where(a_80 == max(a_80))])

a_100 = (BJs_100_peak * var_spin_100_peak)
print(max(a_100))
print(BJs_100_peak[np.where(a_100 == max(a_100))])

a_120 = (BJs_120_peak * var_spin_120_peak)
print(max(a_120))
print(BJs_120_peak[np.where(a_120 == max(a_120))])

In [ ]:
#plot average magnetisation
plt.plot(1/BJs_10_peak, (mean_spin_10_peak), "o", color='red', label='10x10')
plt.plot(1/BJs_30_peak, (mean_spin_30_peak), "o", color='blue', label='30x30')
plt.plot(1/BJs_50_peak, (mean_spin_50_peak), "o", color='green', label='50x50')
plt.plot(1/BJs_80_peak, (mean_spin_80_peak), "o", color='cyan', label='80x80')
plt.plot(1/BJs_100_peak, (mean_spin_100_peak), "o", color='magenta', label='100x100')
plt.plot(1/BJs_120_peak, (mean_spin_120_peak), "o", color='black', label='120x120')
plt.xlabel(r'$T$')
plt.ylabel(r'$<m>$')
plt.legend()
#plt.savefig('M_peak')
plt.show()
plt.close()

In [ ]:
#plot average Energy 
plt.plot(1/BJs_10_peak, (mean_E_10_peak), "o", color='red', label='10x10')
plt.plot(1/BJs_30_peak, (mean_E_30_peak), "o", color='blue', label='30x30')
plt.plot(1/BJs_50_peak, (mean_E_50_peak), "o", color='green', label='50x50')
plt.plot(1/BJs_80_peak, (mean_E_80_peak), "o", color='cyan', label='80x80')
plt.plot(1/BJs_100_peak, (mean_E_100_peak), "o", color='magenta', label='100x100')
plt.plot(1/BJs_120_peak, (mean_E_120_peak), "o", color='black', label='120x120')
plt.xlabel(r'$T$')
plt.ylabel(r'$<E>$')
plt.legend()
#plt.savefig('E_peak')
plt.show()
plt.close()

In [ ]:
#plot average specific heat 
plt.plot(1/BJs_10_peak, (((BJs_10_peak)**2) * (var_E_10_peak)), "o", color='red', label='10x10')
plt.plot(1/BJs_30_peak, (((BJs_30_peak)**2) * (var_E_30_peak)), "o", color='blue', label='30x30')
plt.plot(1/BJs_50_peak, (((BJs_50_peak)**2) * (var_E_50_peak)), "o", color='green', label='50x50')
plt.plot(1/BJs_80_peak, (((BJs_80_peak)**2) * ((var_E_80_peak))), "o", color='cyan', label='80x80')
plt.plot(1/BJs_100_peak, (((BJs_100_peak)**2) * ((var_E_100_peak))), "o", color='magenta', label='100x100')
plt.plot(1/BJs_120_peak, (((BJs_120_peak)**2) * ((var_E_120_peak))), "o", color='black', label='120x120')
plt.xlabel(r'$T$')
plt.ylabel(r'$C_{H}$')
plt.legend()
#plt.savefig('C_H_peak')
plt.show()
plt.close()

#find peak points for specific heat for different lattice sizes
b_10 = (((BJs_10_peak)**2) * (var_E_10_peak))
print(max(b_10))
print(BJs_10_peak[np.where(b_10 == max(b_10))])

b_30 = (((BJs_30_peak)**2) * (var_E_30_peak))
print(max(b_30))
print(BJs_30_peak[np.where(b_30 == max(b_30))])

b_50 = (((BJs_50_peak)**2) * (var_E_50_peak))
print(max(b_50))
print(BJs_50_peak[np.where(b_50 == max(b_50))])

b_80 = (((BJs_80_peak)**2) * ((var_E_80_peak)))
print(max(b_80))
print(BJs_80_peak[np.where(b_80 == max(b_80))])

b_100 = (((BJs_100_peak)**2) * ((var_E_100_peak)))
print(max(b_100))
print(BJs_100_peak[np.where(b_100 == max(b_100))])

b_120 = (((BJs_120_peak)**2) * ((var_E_120_peak)))
print(max(b_120))
print(BJs_120_peak[np.where(b_120 == max(b_120))])

In [ ]:
#plot first comulant 
plt.plot(1/BJs_10_peak, ((mean_mE_10_peak - mean_E_10_peak*mean_spin_10_peak)/mean_spin_10_peak) , "o", color='red', label='10x10')
plt.plot(1/BJs_30_peak, ((mean_mE_30_peak - mean_E_30_peak*mean_spin_30_peak)/mean_spin_30_peak), "o", color='blue', label='30x30')
plt.plot(1/BJs_50_peak, ((mean_mE_50_peak - mean_E_50_peak*mean_spin_50_peak)/mean_spin_50_peak), "o", color='green', label='50x50')
plt.plot(1/BJs_80_peak, ((mean_mE_80_peak - mean_E_80_peak*mean_spin_80_peak)/mean_spin_80_peak), "o", color='cyan', label='80x80')
plt.plot(1/BJs_100_peak, ((mean_mE_100_peak - mean_E_100_peak*mean_spin_100_peak)/mean_spin_100_peak), "o", color='magenta', label='100x100')
plt.plot(1/BJs_120_peak, ((mean_mE_120_peak - mean_E_120_peak*mean_spin_120_peak)/mean_spin_120_peak), "o", color='black', label='120x120')
plt.xlabel(r'$T$')
plt.ylabel(r'$v_{1}$')
plt.legend()
#plt.savefig('v_1_peak')
plt.show()
plt.close()

#find minimum points for first comulant at different lattice sizes  
v_1_10 = (((mean_mE_10_peak - mean_E_10_peak*mean_spin_10_peak)/mean_spin_10_peak))
print(max(v_1_10))
print(BJs_10_peak[np.where(v_1_10 == max(v_1_10))])

v_1_30 = (((mean_mE_30_peak - mean_E_30_peak*mean_spin_30_peak)/mean_spin_30_peak))
print(max(v_1_30))
print(BJs_30_peak[np.where(v_1_30 == max(v_1_30))])

v_1_50 = (((mean_mE_50_peak - mean_E_50_peak*mean_spin_50_peak)/mean_spin_50_peak))
print(max(v_1_50))
print(BJs_50_peak[np.where(v_1_50 == max(v_1_50))])

v_1_80 = (((mean_mE_80_peak - mean_E_80_peak*mean_spin_80_peak)/mean_spin_80_peak))
print(max(v_1_80))
print(BJs_80_peak[np.where(v_1_80 == max(v_1_80))])

v_1_100 = (((mean_mE_100_peak - mean_E_100_peak*mean_spin_100_peak)/mean_spin_100_peak))
print(max(v_1_100))
print(BJs_100_peak[np.where(v_1_100 == max(v_1_100))])

v_1_120 = (((mean_mE_120_peak - mean_E_120_peak*mean_spin_120_peak)/mean_spin_120_peak))
print(max(v_1_120))
print(BJs_120_peak[np.where(v_1_120 == max(v_1_120))])


In [ ]:
#plot second comulant 
plt.plot(1/BJs_10_peak, ((mean_m2E_10_peak - mean_E_10_peak*mean_spin_squared_10_peak)/mean_spin_squared_10_peak) , "o", color='red', label='10x10')
plt.plot(1/BJs_30_peak, ((mean_m2E_30_peak - mean_E_30_peak*mean_spin_squared_30_peak)/mean_spin_squared_30_peak), "o", color='blue', label='30x30')
plt.plot(1/BJs_50_peak, ((mean_m2E_50_peak - mean_E_50_peak*mean_spin_squared_50_peak)/mean_spin_squared_50_peak), "o", color='green', label='50x50')
plt.plot(1/BJs_80_peak, ((mean_m2E_80_peak - mean_E_80_peak*mean_spin_squared_80_peak)/mean_spin_squared_80_peak), "o", color='cyan', label='80x80')
plt.plot(1/BJs_100_peak, ((mean_m2E_100_peak - mean_E_100_peak*mean_spin_squared_100_peak)/mean_spin_squared_100_peak), "o", color='magenta', label='100x100')
plt.plot(1/BJs_120_peak, ((mean_m2E_120_peak - mean_E_120_peak*mean_spin_squared_120_peak)/mean_spin_squared_120_peak), "o", color='black', label='120x120')
plt.xlabel(r'$T$')
plt.ylabel(r'$v_{2}$')
plt.legend()
#plt.savefig('v_2_peak')
plt.show()
plt.close()

#find minimum points for second comulant at different lattice sizes  
v_2_10 = (((mean_m2E_10_peak - mean_E_10_peak*mean_spin_squared_10_peak)/mean_spin_squared_10_peak))
print(max(v_2_10))
print(BJs_10_peak[np.where(v_2_10 == max(v_2_10))])

v_2_30 = (((mean_m2E_30_peak - mean_E_30_peak*mean_spin_squared_30_peak)/mean_spin_squared_30_peak))
print(max(v_2_30))
print(BJs_30_peak[np.where(v_2_30 == max(v_2_30))])

v_2_50 = (((mean_m2E_50_peak - mean_E_50_peak*mean_spin_squared_50_peak)/mean_spin_squared_50_peak))
print(max(v_2_50))
print(BJs_50_peak[np.where(v_2_50 == max(v_2_50))])

v_2_80 = (((mean_m2E_80_peak - mean_E_80_peak*mean_spin_squared_80_peak)/mean_spin_squared_80_peak))
print(max(v_2_80))
print(BJs_80_peak[np.where(v_2_80 == max(v_2_80))])

v_2_100 = (((mean_m2E_100_peak - mean_E_100_peak*mean_spin_squared_100_peak)/mean_spin_squared_100_peak))
print(max(v_2_100))
print(BJs_100_peak[np.where(v_2_100 == max(v_2_100))])

v_2_120 = (((mean_m2E_120_peak - mean_E_120_peak*mean_spin_squared_120_peak)/mean_spin_squared_120_peak))
print(max(v_2_120))
print(BJs_120_peak[np.where(v_2_120 == max(v_2_120))])


In [ ]:
ksi = np.array([2.635029873390992, 19.59207524225383, 48.27821796653929, 129.45979956267337, 163.18805731323187, 275.05661250639156])
log_L = np.array([np.log(10), np.log(30), np.log(50), np.log(80), np.log(100), np.log(120)])
BJs_ksi = np.array([0.4176667, 0.4251667, 0.4331667, 0.4386667, 0.4361667, 0.4386667])


C_H = np.array([1.4206702422919204, 1.9810084739516498, 2.239568110641365,2.599293976452186, 2.5905579578339597, 2.9199998676219034])
log_L_C_H = np.array([np.log(10), np.log(30), np.log(50), np.log(80), np.log(100), np.log(120)])
BJs_C_H = np.array([0.4271667, 0.4386667, 0.4371667, 0.4386667, 0.4396667, 0.4416667])


v_1 = np.array([0.047529056797738715, 0.008933623441613947, 0.003898298532985607, 0.0016667429363252922, 0.001029124334373555, 0.000744814444682752])
log_L_v1 = np.array([np.log(10), np.log(30), np.log(50), np.log(80), np.log(100), np.log(120)])
BJs_v_1 = np.array([0.4461667, 0.4496667, 0.4501667, 0.4501667, 0.4496667, 0.4501667])


v_2 = np.array([0.07800116383577999, 0.016525892257835826, 0.007427110004031646, 0.003249027200857729, 0.0020324463732439746, 0.0014790924870641815])
log_L_v2 = np.array([np.log(10), np.log(30), np.log(50), np.log(80), np.log(100), np.log(120)])
BJs_v_2 = np.array([0.4476667, 0.4496667, 0.4501667, 0.4501667,0.4496667, 0.4501667])

In [ ]:
#plot log(\chi_{max}) as a function of log(L)
a_ksi, b_ksi = np.polyfit(log_L, np.log(ksi), 1)
plt.plot(log_L, a_ksi*log_L+b_ksi, label="linear fit")
plt.scatter(log_L, np.log(ksi), color="cyan", label="calculated values")
plt.xlabel(r'$log(L)$')
plt.ylabel(r'$log(\chi_{max})$')
plt.legend()
plt.savefig('log_Xmax_logL')
plt.show()
plt.close()
print(a_ksi)

In [ ]:
#plot log(\C_{max}) as a function of log(L)
a_C_H, b_C_H = np.polyfit(log_L, np.log(C_H), 1)
plt.plot(log_L, a_C_H* log_L + b_C_H, label="linear fit")
plt.scatter(log_L, np.log(C_H), color="cyan", label="calculated values")
plt.xlabel(r'$log(L)$')
plt.ylabel(r'$log(C_{H max})$')
plt.legend()
plt.savefig('log_C_Hmax_logL')
plt.show()
plt.close()
print(a_C_H)

In [ ]:
#plot log(v_{1max}) as a function of log(L)
a_v1, b_v1 = np.polyfit(log_L, np.log(v_1), 1)
plt.plot(log_L, a_v1 * log_L+ b_v1, label="linear fit")
plt.scatter(log_L, np.log(v_1), color="cyan", label="calculated values")
plt.xlabel(r'$log(L)$')
plt.ylabel(r'$log(v_{1max})$')
plt.legend()
plt.savefig('log_v1_logL')
plt.show()
plt.close()
print(a_v1)

In [ ]:
#plot log(v_{2max}) as a function of log(L)
a_v2, b_v2 = np.polyfit(log_L, np.log(v_2), 1)
plt.plot(log_L, a_v2 * log_L+ b_v2, label="linear fit")
plt.scatter(log_L, np.log(v_2), color="cyan", label="calculated values")
plt.xlabel(r'$log(L)$')
plt.ylabel(r'$log(v_{1max})$')
plt.legend()
plt.savefig('log_v2_logL')
plt.show()
plt.close()
print(a_v2)